# Torch Hub Inference Tutorial

In this tutorial you'll learn:
- how to load a pretrained model using Torch Hub 
- run inference to classify the action in a demo video


NOTE: Currently this tutorial will only work with a local clone of the PyTorchVideo GitHub repo. 

### Import modules

In [1]:
import json 
import torch
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample
) 

### Setup 

Download the id to label mapping for the Kinetics 400 dataset on which the torch hub models were trained. 
This will be used to get the category label names from the predicted class ids.

In [ ]:
!wget https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json 

In [2]:
with open("kinetics_classnames.json", "r") as f:
    kinetics_classnames = json.load(f)

# Create an id to label name mapping
kinetics_id_to_classname = {}
for k, v in kinetics_classnames.items():
    kinetics_id_to_classname[v] = str(k).replace('"', "")

### Load Model using Torch Hub API

PyTorchVideo provides several pretrained models through Torch Hub. Available models are described in [model zoo documentation](https://github.com/facebookresearch/pytorchvideo/blob/master/docs/source/model_zoo.md#kinetics-400). 

Here we are selecting the `slow_r50` model which was trained using a 8x8 setting on the Kinetics 400 dataset. 


In [3]:
# Device on which to run the model
device = "cuda:1"

# Pick a pretrained model 
model_name = "slow_r50"

# Local path to the parent folder of hubconf.py in the pytorchvideo codebase
path = '../' 
model = torch.hub.load(path, source="local", model=model_name, pretrained=True)

# Set to eval mode and move to desired device
model = model.eval()
model = model.to(device)

### Define the transformations for the input required by the model

Before passing the video into the model we need to apply some input transforms and sample a clip of the correct duration. 

In [4]:
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 8
sampling_rate = 8
frames_per_second = 30

# Note that this transform is specific to the slow_R50 model. 
# If you want to try another of the torch hub models you will need to modify this transform
transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size=(crop_size, crop_size))
        ]
    ),
)

# The duration of the input clip is also specific to the model.
clip_duration = (num_frames * sampling_rate)/frames_per_second

### Load an example video
We can test the classification of an example video from the kinetics validation set such as this [archery video](https://www.youtube.com/watch?v=3and4vWkW4s).

In [ ]:
# Download the example video file
# !wget https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4 

In [ ]:
# Load the example video
video_path = "archery.mp4"  

# Select the duration of the clip to load by specifying the start and end duration
# The start_sec should correspond to where the action occurs in the video
start_sec = 0

In [5]:
def get_inp(video_path, start_sec, clip_duration):
    end_sec = start_sec + clip_duration 

    # Initialize an EncodedVideo helper class
    video = EncodedVideo.from_path(video_path)

    # Load the desired clip
    video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

    # Apply a transform to normalize the video input
    video_data = transform(video_data)

    # Move the inputs to the desired device
    inputs = video_data["video"]
    inputs = inputs.to(device)
    return inputs

In [ ]:
# clip_duration

In [ ]:
inputs = get_inp(video_path, start_sec, clip_duration)

### Get model predictions

In [42]:
def get_preds(inputs, model, kinetics_id_to_classname):
    # Pass the input clip through the model 
    preds = model(inputs[None, ...])

    # Get the predicted classes 
    post_act = torch.nn.Softmax(dim=1)
    preds = post_act(preds)

    preds1 = preds.topk(k=5)
    pred_classes = preds1.indices
    pred_scores = preds1.values.tolist()[0]
    # Map the predicted classes to the label names
    pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes[0]]
#     print("Predicted labels: %s" % ", ".join(pred_class_names))
    return (pred_class_names, pred_scores)

In [7]:
def full_pipe(video_path, start_sec, clip_duration, model, kinetics_id_to_classname):
    inputs = get_inp(video_path, start_sec, clip_duration)
    pred_cls = get_preds(inputs, model, kinetics_id_to_classname)
    return pred_cls

# Visualize

In [8]:
from IPython.display import HTML

In [9]:
video_path = '../../temp_dir2/archery.mp4'
!cp {video_path} ./yt_vids/
start_sec = 0
pred_class_names = full_pipe(video_path, start_sec, clip_duration, model, kinetics_id_to_classname)

Predicted labels: archery, throwing axe, playing paintball, stretching arm, riding or walking with horse


In [11]:
from pathlib import Path

In [12]:
video_path_html = Path('yt_vids') / Path(video_path).name

In [13]:
video_path_html

PosixPath('yt_vids/archery.mp4')

In [15]:
def display_html(video_path_html, start_sec, end_sec, pred_class_names):
    return HTML(f"""
<video width="320" height="240" controls>
  <source src="{video_path_html}#t={start_sec},{end_sec}" type="video/mp4">
</video>
<br>
<span> Predicted Classes: {pred_class_names} </span>
""")

In [46]:
def viz(video_path, start_sec, clip_duration, model, kinetics_id_to_classname):
#     video_path = '../../temp_dir2/archery.mp4'
    !cp {video_path} ./yt_vids/
#     start_sec = 2
    video_path_html = Path('yt_vids') / Path(video_path).name
    pred_class_names, pred_scores = full_pipe(video_path, start_sec, clip_duration, model, kinetics_id_to_classname)
    pred_cls_scores = [(x, y) for x, y in zip(pred_class_names, pred_scores)]
    print(pred_class_names, pred_scores)
    return display_html(video_path_html, start_sec, start_sec+clip_duration, pred_cls_scores)

In [43]:
viz(video_path, start_sec, clip_duration, model, kinetics_id_to_classname)

['archery', 'throwing axe', 'playing paintball', 'stretching arm', 'riding or walking with horse'] [1.0, 2.665544691409166e-21, 4.776629300621033e-24, 1.6704025143984983e-24, 1.4239494007063247e-25]


In [44]:
viz(video_path, 4, clip_duration, model, kinetics_id_to_classname)

['archery', 'throwing axe', 'stretching arm', 'playing paintball', 'golf driving'] [1.0, 6.124903375269271e-22, 4.0900786258189156e-25, 2.903209968671495e-26, 2.6353279045492036e-26]


In [49]:
video_path = '/home/arkas/Datasets/vsitu_data/vsitu_trimmed_videos_mp4/v_23v-gPJiaZs_seg_100_110_trimmed.mp4'
viz(video_path, 6, clip_duration, model, kinetics_id_to_classname)

['barbequing', 'extinguishing fire', 'archery', 'smoking', 'cooking chicken'] [0.25146159529685974, 0.04925118386745453, 0.04196379333734512, 0.04175801947712898, 0.03815781697630882]
